<a href="https://colab.research.google.com/github/hongwon2/BigQuery-Practice/blob/main/%EA%B3%A0%EA%B0%9D_%EB%A7%88%EC%8A%A4%ED%84%B0_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC_%EC%97%B0%EB%A0%B9%EB%B3%84%EB%A1%9C_%EC%A7%91%EA%B3%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#권한 설정

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
%load_ext google.colab.data_table

#고객 마스터 데이터 셋

In [3]:
%%bigquery --project ringed-marker-347607
select *
from ch5.mst_users

,user_id,sex,birth_date,register_date,register_device,withdraw_date
0,U001,M,1977-06-17,2016-10-01,pc,None
1,U009,M,2004-10-23,2016-10-15,pc,None
2,U003,M,1965-01-06,2016-10-01,pc,None
3,U010,F,1987-03-18,2016-10-16,pc,None
4,U004,F,1954-05-21,2016-10-05,pc,None
5,U005,M,1987-11-23,2016-10-05,sp,None
6,U008,F,2006-12-09,2016-10-10,sp,None
7,U007,F,1950-07-18,2016-10-10,app,None
8,U006,F,1950-01-21,2016-10-10,pc,2016-10-10
9,U002,F,1953-06-12,2016-10-01,sp,2016-10-10


# 사용자의 생일을 사용하여 목표 날짜에 해당하는 생일을 구하는 쿼리

In [4]:
%%bigquery --project ringed-marker-347607
with
mst_users_with_int_birth_date as(
  select *,
  --특정날짜 (2022-06-21)의 정수 표현
  20220621 as int_specific_date,
  cast(replace(substr(birth_date,1,10),"-","") as int64) as int_birth_date
  from `ringed-marker-347607.ch5.mst_users`
),
mst_users_with_age as (
  select *,
  floor((int_specific_date - int_birth_date)/ 10000) as age
  from
  mst_users_with_int_birth_date
)
select
user_id,sex,birth_date,age
from mst_users_with_age

,user_id,sex,birth_date,age
0,U001,M,1977-06-17,45.0
1,U009,M,2004-10-23,17.0
2,U003,M,1965-01-06,57.0
3,U010,F,1987-03-18,35.0
4,U004,F,1954-05-21,68.0
5,U005,M,1987-11-23,34.0
6,U008,F,2006-12-09,15.0
7,U007,F,1950-07-18,71.0
8,U006,F,1950-01-21,72.0
9,U002,F,1953-06-12,69.0


# 성별과 연령별로 카테고리 구분하기

In [6]:
%%bigquery --project ringed-marker-347607
with
mst_users_with_int_birth_date as(
  select *,
  --특정날짜 (2022-06-21)의 정수 표현
  20220621 as int_specific_date,
  cast(replace(substr(birth_date,1,10),"-","") as int64) as int_birth_date
  from `ringed-marker-347607.ch5.mst_users`
),
mst_users_with_age as (
  select *,
  floor((int_specific_date - int_birth_date)/ 10000) as age
  from
  mst_users_with_int_birth_date
),
mst_users_with_category as (
  select
  user_id,
  sex,
  age,
  concat(case
           when 20 <= age then sex else "" end,
         case
           when age between 4 and 12 then 'C'
           when age between 13 and 19 then 'T'
           when age between 20 and 34 then '1'
           when age between 35 and 49 then '2'
           when age >= 50 then '3'
         end
        ) as category
  from
    mst_users_with_age
)
select *
from 
  mst_users_with_category

,user_id,sex,age,category
0,U001,M,45.0,M2
1,U009,M,17.0,T
2,U003,M,57.0,M3
3,U010,F,35.0,F2
4,U004,F,68.0,F3
5,U005,M,34.0,M1
6,U008,F,15.0,T
7,U007,F,71.0,F3
8,U006,F,72.0,F3
9,U002,F,69.0,F3


#카테고리 별 사람 수를 구하는 쿼리

In [7]:
%%bigquery --project ringed-marker-347607
with
mst_users_with_int_birth_date as(
  select *,
  --특정날짜 (2022-06-21)의 정수 표현
  20220621 as int_specific_date,
  cast(replace(substr(birth_date,1,10),"-","") as int64) as int_birth_date
  from `ringed-marker-347607.ch5.mst_users`
),
mst_users_with_age as (
  select *,
  floor((int_specific_date - int_birth_date)/ 10000) as age
  from
  mst_users_with_int_birth_date
),
mst_users_with_category as (
  select
  user_id,
  sex,
  age,
  concat(case
           when 20 <= age then sex else "" end,
         case
           when age between 4 and 12 then 'C'
           when age between 13 and 19 then 'T'
           when age between 20 and 34 then '1'
           when age between 35 and 49 then '2'
           when age >= 50 then '3'
         end
        ) as category
  from
    mst_users_with_age
)
select
  category,
  count(1) as user_count
from 
  mst_users_with_category
group by
  category

,category,user_count
0,M2,1
1,T,2
2,M3,1
3,F2,1
4,F3,4
5,M1,1


#액션 로그 테이블에서 구매 액션을 한 사람과 카테고리를 조인하여 분석하기

## 액션 로그 테이블

In [9]:
%%bigquery --project ringed-marker-347607
select *
from ch5.action_log

,session,user_id,action,category,products,amount,stamp
0,989004ea,U001,view,None,None,NaN,2016-11-03 18:00:00
1,989004ea,U001,review,drama,D001,NaN,2016-11-03 18:00:00
2,989004ea,U001,add_cart,drama,D001,NaN,2016-11-03 18:00:00
3,989004ea,U001,add_cart,drama,D001,NaN,2016-11-03 18:00:00
4,989004ea,U001,add_cart,drama,D001,NaN,2016-11-03 18:00:00
5,989004ea,U001,add_cart,drama,D001,NaN,2016-11-03 18:00:00
6,989004ea,U001,add_cart,drama,D001,NaN,2016-11-03 18:00:00
7,989004ea,U001,add_cart,drama,D002,NaN,2016-11-03 18:01:00
8,989004ea,U001,add_cart,drama,"D001,D002",NaN,2016-11-03 18:02:00
9,47db0370,U002,add_cart,drama,D001,NaN,2016-11-03 19:00:00


In [8]:
%%bigquery --project ringed-marker-347607
with
mst_users_with_int_birth_date as(
  select *,
  --특정날짜 (2022-06-21)의 정수 표현
  20220621 as int_specific_date,
  cast(replace(substr(birth_date,1,10),"-","") as int64) as int_birth_date
  from `ringed-marker-347607.ch5.mst_users`
),
mst_users_with_age as (
  select *,
  floor((int_specific_date - int_birth_date)/ 10000) as age
  from
  mst_users_with_int_birth_date
),
mst_users_with_category as (
  select
  user_id,
  sex,
  age,
  concat(case
           when 20 <= age then sex else "" end,
         case
           when age between 4 and 12 then 'C'
           when age between 13 and 19 then 'T'
           when age between 20 and 34 then '1'
           when age between 35 and 49 then '2'
           when age >= 50 then '3'
         end
        ) as category
  from
    mst_users_with_age
)
select
  p.category as product_category,
  u.category as user_category,
  count(*) as purchase_count
from 
  `ringed-marker-347607.ch5.action_log` as p
  join
  mst_users_with_category as u
  on p.user_id = u.user_id
where
  -- 구매 로그만 선택하기
  action = 'purchase'
group by
p.category , u.category
order by
p.category, u.category

,product_category,user_category,purchase_count
0,action,M2,1
1,drama,F3,2
2,drama,M2,2
